<a href="https://colab.research.google.com/github/tGhattas/DL/blob/master/DL_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install --upgrade pip
# !pip install tensorflow

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
main_folder = "/content/drive/MyDrive/Colab Notebooks/DL-HW1"
os.makedirs(main_folder, exist_ok=True)
os.chdir(main_folder)

Mounted at /content/drive


In [3]:
!pwd

/content/drive/MyDrive/Colab Notebooks/DL-HW1


In [4]:
# ! git clone https://github.com/tGhattas/DL.git

In [5]:
try:
  os.chdir("./DL/DL-HW1")
except:
  print("warning: git folder was not loaded")

In [6]:
!git reset --hard
!git pull

HEAD is now at d36a599 model refac
Already up to date.


Import TensorFlow into your program:

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from data import prepare_data

In [8]:
(x_train, y_train), (x_test, y_test) = prepare_data()


Use `tf.data` to batch and shuffle the dataset:

In [9]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Build the `tf.keras` model using the Keras [model subclassing API](https://www.tensorflow.org/guide/keras#model_subclassing):

In [10]:
from models import MLP
model = MLP()

Choose an optimizer and loss function for training: 

In [11]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [12]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Use `tf.GradientTape` to train the model:

In [13]:
@tf.function
def train_step(peptides, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(peptides, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

Test the model:

In [14]:
@tf.function
def test_step(peptides, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(peptides, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [15]:
EPOCHS = 100

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for peptides, labels in train_ds:
    train_step(peptides, labels)

  for test_peptides, test_labels in test_ds:
    test_step(test_peptides, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.4184456765651703, Accuracy: 89.42053985595703, Test Loss: 0.3957914412021637, Test Accuracy: 91.04966735839844
Epoch 2, Loss: 0.386785626411438, Accuracy: 92.1859359741211, Test Loss: 0.38700437545776367, Test Accuracy: 92.01382446289062
Epoch 3, Loss: 0.373731791973114, Accuracy: 93.78240966796875, Test Loss: 0.38484081625938416, Test Accuracy: 92.28669738769531
Epoch 4, Loss: 0.36370405554771423, Accuracy: 94.84671783447266, Test Loss: 0.3836365044116974, Test Accuracy: 92.5413818359375
Epoch 5, Loss: 0.35535022616386414, Accuracy: 95.7291030883789, Test Loss: 0.3853078782558441, Test Accuracy: 92.3776626586914
Epoch 6, Loss: 0.3489743173122406, Accuracy: 96.4522933959961, Test Loss: 0.38370996713638306, Test Accuracy: 92.77787780761719
Epoch 7, Loss: 0.3452564477920532, Accuracy: 96.80706024169922, Test Loss: 0.3881705403327942, Test Accuracy: 92.05020904541016
Epoch 8, Loss: 0.34165817499160767, Accuracy: 97.16637420654297, Test Loss: 0.38851454854011536, Test Accu